<a href="https://colab.research.google.com/github/jianfeiZhao/Python_FromBasicToAdvance/blob/master/01Profiler%26Decorator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


##Profiler & Decorator

**1. Profiler**

In [1]:
def is_primer(n):
  # judge prime number

  if n<2: return False
  for i in range(2,n):  # start from 2
    if n % i == 0: return False
  return True

# test cases
assert not is_primer(0)
assert not is_primer(1)
assert is_primer(2)
assert is_primer(5)
assert not is_primer(10)
assert is_primer(101)

print('test cases pass')

def get_primers1(n):
  # get prime numbers from 1 to n
  
  results = []

  for i in range(n+1):
    if is_primer(i):
      results.append(i)

  return results

test cases pass


In [2]:
%prun x = get_primers1(30000)

All the programming languages have profiler:

- observe performance of each function/method.
- observe performance of each line.

In [3]:
!pip install line_profiler
%load_ext line_profiler

     |████████████████████████████████| 71kB 2.0MB/s 


In [0]:
%lprun -f is_primer get_primers1(10000)

is_primer take 99% time, let's optimize it.

**Version 2**

In [5]:
def is_primer2(n):
  # judge prime number
  
  if n<2: return False

  root = int(n**0.5) + 1

  for i in range(2,root):  # start from 2
    if n % i == 0: return False
  return True

# test cases
assert not is_primer2(0)
assert not is_primer2(1)
assert is_primer2(2)
assert is_primer2(5)
assert not is_primer2(10)
assert is_primer2(101)

print('test cases pass')

def get_primers2(n):
  # get prime numbers from 1 to n
  
  results = []

  for i in range(n+1):
    if is_primer2(i):
      results.append(i)

  return results

test cases pass


In [6]:
%time x1 = get_primers1(20000)
%time x2 = get_primers2(20000)

CPU times: user 1.51 s, sys: 2.95 ms, total: 1.51 s
Wall time: 1.51 s
CPU times: user 26.8 ms, sys: 0 ns, total: 26.8 ms
Wall time: 26.7 ms


In [0]:
%lprun -f is_primer2 get_primers2(10000)

It seems better now, but is_primer also take 88% time, can we further optimize it?

Idea: 1/2 + 1/3 + 1/5 = 21/30 numbers in the world can be divided evenly by 2/3/5.

**Version 3**

In [8]:
def is_primer3(n):
  # judge prime number
  
  if n < 2: return False

  if n != 2 and n % 2 == 0: return False
  if n != 3 and n % 3 == 0: return False
  if n != 5 and n % 5 == 0: return False

  root = int(n**0.5) + 1

  for i in range(7,root):  # start from 2
    if n % i == 0: return False
  return True

# test cases
assert not is_primer3(0)
assert not is_primer3(1)
assert is_primer3(2)
assert is_primer3(5)
assert not is_primer3(10)
assert is_primer3(101)

print('test cases pass')

def get_primers3(n):
  # get prime numbers from 1 to n
  
  results = []

  for i in range(n+1):
    if is_primer3(i):
      results.append(i)

  return results

test cases pass


In [9]:
%time x1 = get_primers2(80000)
%time x2 = get_primers3(80000)

CPU times: user 152 ms, sys: 2.03 ms, total: 154 ms
Wall time: 156 ms
CPU times: user 115 ms, sys: 1.94 ms, total: 117 ms
Wall time: 117 ms


In [0]:
%lprun -f is_primer3 get_primers3(30000)

Now it seems better!

**2. Decorator**

   Function-oriented: let functions to be the variables

In [22]:
def get_primers(n, is_primer_fun):
  # get prime numbers from 1 to n
  
  results = []

  for i in range(n+1):
    if is_primer_fun(i):
      results.append(i)

  return results

%time x1 = get_primers(30000, is_primer)
%time x2 = get_primers(30000, is_primer2)
%time x3 = get_primers(30000, is_primer3)


CPU times: user 3.3 s, sys: 2.91 ms, total: 3.3 s
Wall time: 3.3 s
CPU times: user 42.5 ms, sys: 0 ns, total: 42.5 ms
Wall time: 42.4 ms
CPU times: user 30.8 ms, sys: 0 ns, total: 30.8 ms
Wall time: 30.8 ms


In [0]:
%lprun -f is_primer3 get_primers(60000, is_primer3)

In [0]:
import time
from functools import wraps

cache = {}       # store the variables already used
def get_time(func):
  "my name is _time function"

  global cache

  #@wraps(func)
  def _time(arg1, arg2):
    s = time.time()

    if (arg1, arg2) in cache: result = cache[(arg1, arg2)]
    else:
      result = func(arg1, arg2)
      cache[(arg1, arg2)] = result

    print('used time: {}'.format(time.time()-s))
    return result
  
  return _time

In [67]:
@get_time        # exactly ==> get_primes_f = get_time(get_primes_f)
def get_primers_with_decorator(n, is_primer_fun):
  "get prime numbers from 1 to n"
  
  results = []

  for i in range(n+1):
    if is_primer_fun(i):
      results.append(i)

  return results

t1 = get_primers_with_decorator(80000, is_primer3)
t2 = get_primers_with_decorator(160000, is_primer3)

used time: 3.0994415283203125e-06
used time: 1.430511474609375e-06
